In [1]:
import numpy as np

In [2]:
BATCH_SIZE = 8192

In [504]:
p1s = np.random.rand(BATCH_SIZE, 2)
p2s = np.random.rand(BATCH_SIZE, 2)
p3s = np.random.rand(BATCH_SIZE, 2)
e1s = np.abs(p2s - p1s)
e2s = np.abs(p3s - p2s)
e3s = np.abs(p1s - p3s)
areas = np.concatenate([es[:, 0] * es[:, 1] for es in (e1s, e2s, e3s)]).reshape((BATCH_SIZE, -1), order='F')

In [516]:
areas

array([[0.14259209, 0.05159827, 0.27727633],
       [0.41055818, 0.04199319, 0.10918482],
       [0.05145701, 0.03414722, 0.25771974],
       ...,
       [0.1166764 , 0.0063362 , 0.14681098],
       [0.18243655, 0.02016673, 0.1024437 ],
       [0.02571482, 0.03077929, 0.04438248]])

In [522]:
areas[:, 0:2].max(axis=1)

array([0.14259209, 0.41055818, 0.05145701, ..., 0.1166764 , 0.18243655,
       0.03077929])

In [518]:
areas[:, 1:3].max(axis=1)

array([0.27727633, 0.10918482, 0.25771974, ..., 0.14681098, 0.1024437 ,
       0.04438248])

In [526]:
areas[:, 0:3:2].max(axis=1)

array([0.27727633, 0.41055818, 0.25771974, ..., 0.14681098, 0.18243655,
       0.04438248])

In [527]:
np.c_[areas[:, 0:2].max(axis=1), areas[:, 0:2].max(axis=1), areas[:, 0:2].max(axis=1)]

array([[0.14259209, 0.14259209, 0.14259209],
       [0.41055818, 0.41055818, 0.41055818],
       [0.05145701, 0.05145701, 0.05145701],
       ...,
       [0.1166764 , 0.1166764 , 0.1166764 ],
       [0.18243655, 0.18243655, 0.18243655],
       [0.03077929, 0.03077929, 0.03077929]])

In [3]:
def compute_batch_mean_area(batch_size=BATCH_SIZE, mean=True, idx=1):
    points = np.random.rand(batch_size * 3, 2)
    edges = np.abs(np.roll(points, batch_size, axis=0) - points)
    areas = (edges[:, 0] * edges[:, 1]).reshape((batch_size, -1), order='F')
    areas.sort(axis=1)
    if mean:
        return areas[:, idx].mean()
    else:
        return areas[:, idx]

In [52]:
for i in range(5):
    print(compute_batch_mean_area())

0.10017963923971529
0.10326332247553371
0.10200313995006491
0.10030353675126896
0.10114689444086519


In [572]:
%timeit -n 1000 compute_batch_mean_area(128)
%timeit -n 1000 compute_batch_mean_area(256)
%timeit -n 1000 compute_batch_mean_area(512)
%timeit -n 1000 compute_batch_mean_area(1024)

59.3 µs ± 2.95 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
73.3 µs ± 2.47 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
104 µs ± 1.67 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
158 µs ± 2.52 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [540]:
%timeit -n 1000 compute_batch_mean_area(8192)
%timeit -n 1000 compute_batch_mean_area(16384)
%timeit -n 100 compute_batch_mean_area(65536)

1.26 ms ± 16.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.25 ms ± 24.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
11.6 ms ± 213 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
print(compute_batch_mean_area(100000, idx=0))  # min
print(compute_batch_mean_area(100000, idx=1))  # med
print(compute_batch_mean_area(1000000, idx=2))  # max

0.026810993162328545
0.10187844057132216
0.20462691989410298


In [12]:
(0.026590846337935868+0.20457597729846852)/2

0.1155834118182022

In [573]:
from difflib import SequenceMatcher

def num_leading_digits_in_common(n1, n2):
    s1 = str(n1)
    s2 = str(n2)
    match = SequenceMatcher(None, s1, s2).find_longest_match(0, len(s1), 0, len(s2))
    return match.size - 2 if match.a + match.b == 0 else 0  # For 0.

In [574]:
BATCH_SIZE = 8192
MIN_DIGITS = 10
MIN_MINOR_DIFFS = 3

In [575]:
samples = 0
running_avg = 0
num_minor_diffs = 0

while True:
    samples += 1
    if samples > 1220703:
        break
        
    sample_avg = compute_batch_mean_area(BATCH_SIZE)
    new_avg = running_avg + (sample_avg - running_avg) / samples
    if num_leading_digits_in_common(new_avg, running_avg) >= MIN_DIGITS:
        print(f'[{samples}] new avg. common to {MIN_DIGITS} digits: {running_avg:.010f}, {new_avg:.010f}')
        num_minor_diffs += 1
        if num_minor_diffs >= MIN_MINOR_DIFFS:
            print('[{samples}] final!')
            break
    else:
        if num_minor_diffs > 0:
            print(f'[{samples}] resetting! saw {new_avg:.010f}')
        num_minor_diffs = 0
    running_avg = new_avg

[7015] new avg. common to 10 digits: 0.1017734027, 0.1017734028
[7016] resetting! saw 0.1017735423
[11299] new avg. common to 10 digits: 0.1017746878, 0.1017746878
[11300] resetting! saw 0.1017744889
[11690] new avg. common to 10 digits: 0.1017729892, 0.1017729892
[11691] resetting! saw 0.1017730554


KeyboardInterrupt: 

In [ ]:
(10 ** MIN_DIGITS) / 8192

In [579]:
for i in range(10):
    print(compute_batch_mean_area(1))

0.13063496735483027
0.09001111935435323
0.03923103849534351
0.010676469092827741
0.002196924949194271
0.03696008825357951
0.12488724603978338
0.12337872310805745
0.13476739692449555
0.03995712048649018


In [ ]:
BATCH_SIZE = 8192
samples = 0
running_avg = 0
num_minor_diffs = 0
max_samples = 10 ** (MIN_DIGITS + 1)

while True:    
    for sample_avg in compute_batch_mean_area(BATCH_SIZE, mean=False):
        samples += 1
        delta = (sample_avg - running_avg) / samples
        running_avg += delta
        if samples % 10000000 == 0:
            print(f'[{samples}] running avg: {running_avg:.010f}, delta: {delta:.010f}')
        if samples > max_samples:
            print(f'[{samples}] final! {running_avg:.010f}')
            break

[10000000] running avg: 0.1017655579, delta: -0.0000000046
[20000000] running avg: 0.1017566833, delta: 0.0000000068
[30000000] running avg: 0.1017620801, delta: -0.0000000016
[40000000] running avg: 0.1017552249, delta: -0.0000000003
[50000000] running avg: 0.1017545054, delta: 0.0000000020
[60000000] running avg: 0.1017584850, delta: 0.0000000013
[70000000] running avg: 0.1017542170, delta: 0.0000000014
[80000000] running avg: 0.1017593474, delta: 0.0000000011
[90000000] running avg: 0.1017662421, delta: 0.0000000031
[100000000] running avg: 0.1017691705, delta: 0.0000000001
[110000000] running avg: 0.1017704576, delta: -0.0000000002
[120000000] running avg: 0.1017707650, delta: -0.0000000001
[130000000] running avg: 0.1017731602, delta: -0.0000000007
[140000000] running avg: 0.1017728953, delta: 0.0000000002
[150000000] running avg: 0.1017739369, delta: 0.0000000009
[160000000] running avg: 0.1017732410, delta: -0.0000000000
[170000000] running avg: 0.1017732981, delta: -0.000000000

In [143]:
[11690000000] running avg: 0.0880712192
[11700000000] running avg: 0.0880712118
[11710000000] running avg: 0.0880711937
[11720000000] running avg: 0.0880711876
[11730000000] running avg: 0.0880712180
[11740000000] running avg: 0.0880712113
[11750000000] running avg: 0.0880712169
[11760000000] running avg: 0.0880712352
[11770000000] running avg: 0.0880712686
[11780000000] running avg: 0.0880712331

1220703.125

In [152]:
(10**10) / 8192

1220703.125

In [1]:
import chaospy

In [78]:
distribution = chaospy.J(chaospy.Uniform(0, 1), chaospy.Uniform(0, 1))

In [19]:
q0, q1 = chaospy.variable?

Signature: chaospy.variable(dimensions=1, asarray=False, dtype='i8', allocation=None)
Docstring:
Construct variables that can be used to construct polynomials.

Args:
    dimensions (int):
        Number of dimensions in the array.
    asarray (bool):
        Enforce output as array even in the case where there is only one
        variable.
    dtype (numpy.dtype):
        The data type of the polynomial coefficients.
    allocation (Optional[int]):
        The maximum number of polynomial exponents. If omitted, use
        length of exponents for allocation.

Returns:
    (chaospy.poly.polynomial):
        Polynomial array with unit components in each dimension.

Examples:
    >>> numpoly.variable()
    polynomial(q0)
    >>> q0, q1, q2 = numpoly.variable(3)
    >>> q1+1
    polynomial(q1+1)
    >>> numpoly.polynomial([q2**3, q1+q2, 1])
    polynomial([q2**3, q2+q1, 1])
File:      d:\anaconda\envs\py3\lib\site-packages\numpoly\construct\variable.py
Type:      function


In [ ]:
q0, q1 = chaospy.variable

In [21]:
chaospy.E(chaospy.abs(q1 - q0), distribution)

array(3.)

In [23]:
chaospy.polynomial?

Signature: chaospy.polynomial(poly_like=None, names=None, dtype=None, allocation=None)
Docstring:
Attempt to cast an object into a polynomial array.

Supports various casting options:

==================  =======================================================
``dict``            Keys are tuples that represent polynomial exponents,
                    and values are numpy arrays that represents polynomial
                    coefficients.
``numpoly.ndpoly``  Copy of the polynomial.
``numpy.ndarray``   Constant term polynomial.
``sympy.Poly``      Convert polynomial from ``sympy`` to ``numpoly``,
                    if possible.
``Iterable``        Multivariate array construction.
structured array    Assumes that the input are raw polynomial core and can
                    be used to construct a polynomial without changing the
                    data. Used for developer convenience.
==================  =======================================================

Args:
    poly_like (typin

In [84]:
distribution = chaospy.J(chaospy.Uniform(0, 1), chaospy.Uniform(0, 1))
distribution.sample(10, rule='sobol')

array([[0.5   , 0.75  , 0.25  , 0.375 , 0.875 , 0.625 , 0.125 , 0.1875,
        0.6875, 0.9375],
       [0.5   , 0.25  , 0.75  , 0.375 , 0.875 , 0.125 , 0.625 , 0.3125,
        0.8125, 0.0625]])

In [92]:
np.random.seed(10)

In [94]:
chaospy.generate_samples(order=10, domain=2, rule="halton")

array([[0.125     , 0.625     , 0.375     , 0.875     , 0.0625    ,
        0.5625    , 0.3125    , 0.8125    , 0.1875    , 0.6875    ],
       [0.44444444, 0.77777778, 0.22222222, 0.55555556, 0.88888889,
        0.03703704, 0.37037037, 0.7037037 , 0.14814815, 0.48148148]])

In [27]:
import torch.quasirandom

sobol = torch.quasirandom.SobolEngine(dimension=6)

In [48]:
points = sobol.draw(10)
points

tensor([[0.2827, 0.9331, 0.8929, 0.6397, 0.3942, 0.3448],
        [0.4077, 0.3081, 0.0179, 0.2647, 0.7692, 0.4698],
        [0.9077, 0.8081, 0.5179, 0.7647, 0.2692, 0.9698],
        [0.6577, 0.0581, 0.7679, 0.0147, 0.0192, 0.2198],
        [0.1577, 0.5581, 0.2679, 0.5147, 0.5192, 0.7198],
        [0.2202, 0.4956, 0.9554, 0.8272, 0.7067, 0.6573],
        [0.7202, 0.9956, 0.4554, 0.3272, 0.2067, 0.1573],
        [0.9702, 0.2456, 0.2054, 0.5772, 0.4567, 0.9073],
        [0.4702, 0.7456, 0.7054, 0.0772, 0.9567, 0.4073],
        [0.3452, 0.1206, 0.3304, 0.9522, 0.3317, 0.2823]])

In [59]:
edges = torch.abs(torch.roll(points, 2, dims=1) - points)
edges

tensor([[0.1115, 0.5883, 0.6102, 0.2933, 0.4987, 0.2949],
        [0.3615, 0.1617, 0.3898, 0.0433, 0.7513, 0.2051],
        [0.6385, 0.1617, 0.3898, 0.0433, 0.2487, 0.2051],
        [0.6385, 0.1617, 0.1102, 0.0433, 0.7487, 0.2051],
        [0.3615, 0.1617, 0.1102, 0.0433, 0.2513, 0.2051],
        [0.4865, 0.1617, 0.7352, 0.3317, 0.2487, 0.1699],
        [0.5135, 0.8383, 0.2648, 0.6683, 0.2487, 0.1699],
        [0.5135, 0.6617, 0.7648, 0.3317, 0.2513, 0.3301],
        [0.4865, 0.3383, 0.2352, 0.6683, 0.2513, 0.3301],
        [0.0135, 0.1617, 0.0148, 0.8317, 0.0013, 0.6699]])

In [70]:
areas = torch.stack((edges[:, 0] * edges[:, 1], edges[:, 2] * edges[:, 3], edges[:, 4] * edges[:, 5]), 1)
areas

tensor([[0.0656, 0.1790, 0.1471],
        [0.0585, 0.0169, 0.1541],
        [0.1033, 0.0169, 0.0510],
        [0.1033, 0.0048, 0.1535],
        [0.0585, 0.0048, 0.0515],
        [0.0787, 0.2438, 0.0423],
        [0.4305, 0.1770, 0.0423],
        [0.3398, 0.2537, 0.0829],
        [0.1646, 0.1572, 0.0829],
        [0.0022, 0.0123, 0.0009]])

In [71]:
areas.sort(axis=1)

torch.return_types.sort(
values=tensor([[0.0656, 0.1471, 0.1790],
        [0.0169, 0.0585, 0.1541],
        [0.0169, 0.0510, 0.1033],
        [0.0048, 0.1033, 0.1535],
        [0.0048, 0.0515, 0.0585],
        [0.0423, 0.0787, 0.2438],
        [0.0423, 0.1770, 0.4305],
        [0.0829, 0.2537, 0.3398],
        [0.0829, 0.1572, 0.1646],
        [0.0009, 0.0022, 0.0123]]),
indices=tensor([[0, 2, 1],
        [1, 0, 2],
        [1, 2, 0],
        [1, 0, 2],
        [1, 2, 0],
        [2, 0, 1],
        [2, 1, 0],
        [2, 1, 0],
        [2, 1, 0],
        [2, 0, 1]]))

In [77]:
def compute_batch_mean_area_pseudo(batch_size=BATCH_SIZE, mean=True):
    #points = distribution.sample(batch_size * 3, rule='sobol').T
    #points = np.random.rand(batch_size * 3, 2)
    points = sobol.draw(batch_size * 3)
    edges = torch.abs(torch.roll(points, 2, dims=1) - points)
    areas = torch.stack((edges[:, 0] * edges[:, 1], edges[:, 2] * edges[:, 3], edges[:, 4] * edges[:, 5]), 1)
    areas, _ = areas.sort(axis=1)
    if mean:
        return areas[:, 1].mean().item()
    else:
        return areas[:, 1]

In [82]:
print(compute_batch_mean_area_pseudo(10000000))

0.10139038413763046
